In [45]:
import numpy as np
import itertools as it

#### Read data from file

In [25]:
with open ("data.txt", 'r') as data: #read data 
    dataset = [[int(x) for x in line.split()] for line in data]  

In [26]:
make_R = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
R = make_R(dataset, 15)
print("Number of R: {}".format(len(R)))

Number of R: 10


#### K-optimization

In [27]:
def check_x(S):
    max_k = np.array([])
    opt_k = np.array([])
    for i, x in zip(range(len(S)),S):
        if np.any(S-x > 0): #формуємо множину k-макс. елементів
            continue
        max_k = np.append(max_k, i+1)
        if np.sum(x) == 15: #формуємо множину k-опт. елементів
            opt_k = np.append(opt_k, i+1)
        
    return max_k, opt_k

In [28]:
def k_optimization(Rn):
    I = (Rn == Rn.T)*Rn #формуємо симетричну частину
    P = Rn-I #формуємо асиметричну частину
    N = (Rn == Rn.T)-I #формуємо відношення непорівнюваності
    
    for i in range(1,5):
        if i==1:
            S1=I+P+N
            max_1, opt_1 = check_x(S1)
        elif i==2:
            S2=P+N
            max_2, opt_2 = check_x(S2)
        elif i==3:
            S3=P+I
            max_3, opt_3 = check_x(S3)
        elif i==4:
            S4=P
            max_4, opt_4 = check_x(S4)
            
    parameters = {"1_max": max_1,
                 "1_opt": opt_1,
                 "2_max": max_2,
                 "2_opt": opt_2,
                 "3_max": max_3,
                 "3_opt": opt_3,
                 "4_max": max_4,
                 "4_opt": opt_4}
    
    return parameters

In [29]:
def print_information_k_opt(parameters, num_r):
    print("____R{}____".format(num_r))
    print("1-max: {}".format(parameters["1_max"]))
    print("1-opt: {}".format(parameters["1_opt"]))
    print("2-max: {}".format(parameters["2_max"]))
    print("2-opt: {}".format(parameters["2_opt"]))
    print("3-max: {}".format(parameters["3_max"]))
    print("3-opt: {}".format(parameters["3_opt"]))
    print("4-max: {}".format(parameters["4_max"]))
    print("4-opt: {}".format(parameters["4_opt"])) 

#### Neumann-Morgenstern optimization

In [30]:
def get_pereriz(R):
    verkhi_pererizy = []
    R_t = R.T
    for i in range(len(R)):
        itemindex = (np.where(R_t[i]==1))[0]
        verkhi_pererizy.append(itemindex)
    return verkhi_pererizy 

In [31]:
def make_S0(verkhi_pererizy):
    S, S_delta = dict(), dict()
  
    buffer = []
    for a, index_for_S0 in zip(verkhi_pererizy, range(len(verkhi_pererizy))):
        if a.size == 0:
            buffer.append(index_for_S0)

    S["S0"] = buffer
    S_delta["S0"] = S["S0"]
    return S, S_delta

In [32]:
def make_S(S, S_delta, R, verkhi_pererizy):
    current_i = 0
    l = 0
    while(len(S["S"+str(current_i)])!= len(R)):
        buffer = []
    
        for a, index in zip(verkhi_pererizy, range(len(verkhi_pererizy))):
            inclusion_check = np.in1d(a, S["S"+str(current_i)]) # индекс предыдущего S. Если щас находим S1,то тут должна быть S0

            if np.all(inclusion_check == True):
                buffer.append(index)

        if buffer == S["S"+str(current_i)]: # если поточне = попередньому 
            buffer += list(set(range(len(verkhi_pererizy))) - set(S["S"+str(current_i)]))

        current_i += 1
        l += 1
        S["S"+str(current_i)] = buffer
        delta = list(set(S["S"+str(current_i)]) - set(S["S"+str(current_i-1)]))
        S_delta["S"+str(current_i)+"_d"] = delta 
    return S, S_delta, l

In [33]:
def make_Q(S_delta, Q, verkhi_pererizy, L):
    buffer = []
    for l in range(1,L+1):
        for delta in S_delta["S"+str(l)+"_d"]:
            inclusion_check = np.in1d(verkhi_pererizy[delta], Q["Q"+str(l-1)])

        if np.all(inclusion_check == False):
            buffer.append(delta)

        Q["Q"+str(l)] = Q["Q"+str(l-1)] + buffer
    return Q

In [69]:
def Neumann_Morgenstern_optimization(R):
    verkhi_pererizy = get_pereriz(R) #знаходимо верхні перерізи для всіх альтернатив

    S, S_delta, Q = dict(), dict(), dict()
    S, S_delta = make_S0(verkhi_pererizy) #S0

    S, S_delta, l = make_S(S, S_delta, R, verkhi_pererizy) #STEP 1

    Q["Q0"] = S["S0"] #Q0
    Q = make_Q(S_delta, Q, verkhi_pererizy, l) #STEP 2

    С = Q["Q"+str(l)]
    C0 = [x+1 for x in С]
    return C0

In [66]:
def print_information_NM(s, num_r):
    print("____R{}____".format(num_r))
    print("Xнм: {}".format(s))

#### Check acyclicity

In [72]:
def acyclicity_graph(matrix): #перевірка відношення на аціклічність методом графу
    size = len(matrix)
    E = []
    for i in range(size):
        for j in range(size):
            if matrix[i][j] == 1:
                E.append((i+1,j+1))
    graph = {}
    for k, ir in it.groupby(sorted(E), key=lambda x: x[0]):
        graph[k] = {e for _, e in ir}

    # видаляємо всі вершини які не можуть бути частиною циклу (мають вхідні та вихідні ребра тільки)
    sub_graph = {}
    while True:
        vertex_set = set(graph).intersection(it.chain.from_iterable(graph.values()))
        sub_graph = {k: vertex_set & vs for k, vs in graph.items()
                     if k in vertex_set and vertex_set & vs}

        if sub_graph == graph:
            break
        else:
            graph = sub_graph

    if graph: #чи є підграф
        return False
    else:
        return True

#### Сomputation

In [70]:
for num_r, r_i in zip(range(len(R)),R):
    Rn = np.array(r_i)
    
    if acyclicity_graph(Rn):
        solution = Neumann_Morgenstern_optimization(Rn)
        print_information_NM(list(set(solution)), num_r+1)
    else:
        parameters = k_optimization(Rn)
        print_information_k_opt(parameters, num_r+1)


____R1____
1-max: [3.]
1-opt: [3.]
2-max: [3.]
2-opt: []
3-max: [3.]
3-opt: [3.]
4-max: [3.]
4-opt: []
____R2____
Xнм: [8, 2, 3, 6]
____R3____
Xнм: [2, 5, 6]
____R4____
1-max: []
1-opt: []
2-max: []
2-opt: []
3-max: []
3-opt: []
4-max: []
4-opt: []
____R5____
Xнм: [1, 4, 12, 14]
____R6____
1-max: [ 5.  6.  8. 11. 14.]
1-opt: [ 5.  6.  8. 11. 14.]
2-max: [ 5.  6. 11.]
2-opt: [ 5.  6. 11.]
3-max: []
3-opt: []
4-max: [ 5.  6. 11.]
4-opt: []
____R7____
1-max: [ 4.  7.  8. 13. 14.]
1-opt: [ 4.  7.  8. 13. 14.]
2-max: [ 4.  8. 13.]
2-opt: [ 4.  8. 13.]
3-max: []
3-opt: []
4-max: [ 4.  8. 13.]
4-opt: []
____R8____
1-max: []
1-opt: []
2-max: []
2-opt: []
3-max: []
3-opt: []
4-max: []
4-opt: []
____R9____
1-max: [ 1.  4.  7.  9. 10. 11.]
1-opt: [ 1.  4.  7.  9. 10. 11.]
2-max: [ 1.  4.  7.  9. 10. 11.]
2-opt: [ 1.  4.  7.  9. 10. 11.]
3-max: [ 1.  3.  5.  7. 11.]
3-opt: []
4-max: [ 1.  7. 11.]
4-opt: []
____R10____
Xнм: [1, 2, 11, 7]
